In [ ]:
#########################
# defs and imports
#########################

import logging
import os
import shutil
import json
import threading
import traceback

import numpy as np
from numpy.linalg import inv

from nis_util import *
from simple_detection import *

from skimage.transform import AffineTransform
from skimage.io import imread
from xmlrpc.client import ServerProxy


# file endings as exported by NIS
ND2_SUFFIX = '.nd2'
TIFF_SUFFIX = '.tif'


def copy_lock(src, dst, copyfun=shutil.copy2, lock_ending='lock'):
    lock_file = '.'.join([dst if not os.path.isdir(dst) else os.path.join(dst, src.rsplit(os.sep, 1)[-1]), lock_ending])
    fd = open(lock_file, 'w')
    fd.close()

    copyfun(src, dst)
    os.remove(lock_file)

    
def copy_lock_to_dir(src, dst, copyfun=shutil.copy2, lock_ending='lock'):
    
    if not isinstance(src, list):
        src = [src]
    
    if not os.path.exists(dst):
        os.makedirs(dst)
        
    if os.path.isfile(dst):
        raise ValueError('destination has to be a dirctory')
        
    for s in src:
        copy_lock(s, dst, copyfun, lock_ending)

        
def _pix2unit(x, transform):
    """
    transform a point from pixel coordinates to NIS stage coordinates,
    taking into account offsets, fov, camera rotation or image flipping
    
    Parameters
    ----------
    x: 2-tuple
        point to transform, in pixels  
    transform: AffineTransform
        affine transform pixel -> stage
        
    Returns
    -------
    x_tr: array-like
        transformed point, in units
    """
    res = np.squeeze(transform(x))
    logger.debug('transformed point {} (pixels) to {} (units)'.format(x, res))
    return res
   

def bbox_pix2unit(bbox, transform):
    """
    Parameters
    ----------
    x: 4-tuple
        point to transform, in pixels  
    transform: AffineTransform
        affine transform pixel -> stage
    
    Returns
    -------
    bbox_tr: 4-tuple
        transformed bounding box (ymin, xmin, ymax, xmax - in units)
    """
    
    logger = logging.getLogger(__name__)
      
    # transform bbox
    (ymin, xmin, ymax, xmax) = bbox    
    bbox_tr = np.apply_along_axis(lambda x: _pix2unit(x, transform),
                                  1, 
                                  np.array([[xmin, ymin],
                                            [xmin, ymax],
                                            [xmax, ymin],
                                            [xmax, ymax]], dtype=float)
                                  )
    
    # get new min max
    min_ = np.apply_along_axis(np.min, 0, bbox_tr)
    max_ = np.apply_along_axis(np.max, 0, bbox_tr)
    
    logger.debug('new min: {}, new max: {}'.format(min_, max_))
    
    # NB: we reverse here to preserve original ymin, xmin, ymax, xmax - order
    bbox_tr_arr = np.array([list(reversed(list(min_))), list(reversed(list(max_)))], dtype=float)
    res = bbox_tr_arr.ravel()
    
    logger.debug('bbox: {}, toUnit: {}'.format(bbox, res))
    return tuple(list(res))


class WidgetProgressIndicator:
    """
    thin wrapper around an ipywidgets widget to display progress, e.g. progress bar
    and an optional (text) status widget
    
    Parameters
    ----------
    progress_widget: ipywidgets widget
        widget to display progress, e.g. FloatProgress
    status_widget: ipywidgets widget, optional
        widget to display a status message, e.g. Label
    min: numeric, optional
        value of progress_widget corresponding to 0
    max: numeric, optional
        value of progress_widget corresponding to 1, default 100
    """
    def __init__(self, progress_widget, status_widget=None, min=0, max=100):
        self.progress_widget = progress_widget
        self.status_widget = status_widget
        self.min = min
        self.max = max
    
    def set_progress(self, p):
        """
        update progress
        Parameters
        ----------
        p: float \in 0,1
            percent complete value to set
        """
        self.progress_widget.value = self.min + p * (self.max - self.min)
        
    def set_status(self, status):
        """
        update status
        status: string
            status message
        """
        if self.status_widget is not None:
            self.status_widget.value = status
        

def do_scan(field_def_file, oc_overview, ocs_detail, path_to_nis, save_base_path,
            prefix, server_path_local, server_path_remote, do_plot=True,
            manual_z_overview=None, z_range = 10, z_step=2, z_drive='Ti2 ZDrive', auto_focus_detail=True,
            tiff_export_ov=True, tiff_export_detail=True, dry_run_details=False,
            stitched=True, separate_dirs_on_server=True, re_use_ov=False, progress_indicator=None):
    
    # keep track of all copy threads so we can join on exit
    threads = []
    logger = logging.getLogger(__name__)
    
    try:

        if progress_indicator is not None:
            progress_indicator.set_progress(0.0)
            progress_indicator.set_status('doing overview scan')
            
        if stitched and not np.isscalar(ocs_detail):
            logger.info('Doing multi-channel acquisition, cannot use NIS stitching. Please stitch manually.')

        with open(field_def_file, 'r') as fd:
            field_calib = json.load(fd)

        # user specified manual focus position
        if not manual_z_overview is None:
            field_calib['zpos'] = manual_z_overview
            
        set_position(path_to_nis, pos_z=field_calib['zpos'])
    
        # get field and directions
        # NB: this is not the actual field being scanned, but rather [min+1/2 fov - max-1/2fov]
        (left, right, top, bottom) = tuple(field_calib['bbox'])

        # pixel to world coordinates transformation from 3-point calibration stored in field_calib file
        coords_px = np.array(field_calib['coords_px'], dtype=np.float)
        coords_st = np.array(field_calib['coords_st'], dtype=np.float)
        at = AffineTransform()
        at.estimate(coords_px, coords_st)

        # direction of stage movement (y,x)
        direction = [1 if top<bottom else -1, 1 if left<right else -1]

        # set overview optical configuration
        set_optical_configuration(path_to_nis, oc_overview)

        # get resolution, binning and fov
        (xres, yres, siz, mag) = get_resolution(path_to_nis)
        live_fmt, capture_fmt = get_camera_format(path_to_nis)
        binning_factor = float(capture_fmt.split()[1].split('x')[0])
        fov_x = xres * siz / mag * binning_factor
        fov_y = yres * siz / mag * binning_factor

        logger.debug('overview resolution: {}, {}, {}, {}'.format(xres, yres, siz, mag))

        # do overview scan
        ov_path = os.path.join(save_base_path, prefix + '_overview' + ND2_SUFFIX)
        if not re_use_ov:
            do_large_image_scan(path_to_nis, ov_path, left, right, top, bottom)

        if tiff_export_ov:
            export_nd2_to_tiff(path_to_nis, ov_path)
            tiff_ov_path = ov_path[:-len(ND2_SUFFIX)] + TIFF_SUFFIX
            img = imread(tiff_ov_path)        
        else:
            img = read_bf(ov_path)


        def copy_ov_call():
            # copy to server mount
            _ov_path = ov_path
            _tiff_ov_path = tiff_ov_path
            copy_lock(_ov_path, server_path_local + (os.sep + 'overviews') if separate_dirs_on_server else '')
            copy_lock(_tiff_ov_path, server_path_local + (os.sep + 'overviews') if separate_dirs_on_server else '')


        # copy in separate thread
        copy_ov_thread = threading.Thread(target=copy_ov_call)
        threads.append(copy_ov_thread)
        copy_ov_thread.start()

        # we want 4x downsampling for detection
        ds = max(0, int(round(2 - np.log2(binning_factor))))

        if ds != 0:
            img = list(pyramid_gaussian(img, ds))[-1]

        # NB: we have to wait for copy to complete before we initialize the detection on server
        copy_ov_thread.join()

        logger.info('finished overview, detecting wings...')

        flt = {
            'area': (15000, 80000)
        }

        _suffix = TIFF_SUFFIX if tiff_export_ov else ND2_SUFFIX
        if separate_dirs_on_server:
            remote_path = '/'.join([server_path_remote, 'overviews', prefix + '_overview' + _suffix])
        else:
            remote_path = '/'.join([server_path_remote, prefix + '_overview' + _suffix])

        if progress_indicator is not None:
            progress_indicator.set_status('detecting wings')
            
        with ServerProxy("http://eco-gpu:8000/") as proxy:
            bboxes = proxy.detect_bbox(remote_path, binning_factor, flt)

        if do_plot:
            plt.figure()
            plt.imshow(img)

        bboxes_scaled = []
        for bbox in bboxes[0]:
            # upsample bounding boxes if necessary
            bbox_scaled = np.array(tuple(bbox)) * binning_factor * 2**ds
            logger.debug('bbox: {}, upsampled: {}'.format(bbox, bbox_scaled))
            bboxes_scaled.append(bbox_scaled)
            
            # plot bbox
            if do_plot:
                minr, minc, maxr, maxc = tuple(list(bbox))
                rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                      fill=False, edgecolor='red', linewidth=2)
                plt.gca().add_patch(rect)
        if do_plot:
            plt.show()

        # use scaled bboxes from here on
        bboxes = bboxes_scaled

        # pixels to units
        bboxes = [bbox_pix2unit(b, at) for b in bboxes]

        # expand bounding boxes
        bboxes = [scale_bbox(bbox, expand_factor=.2) for bbox in bboxes]

        logger.info('detected {} wings:'.format(len(bboxes)))

        # scan the individual wings
        for idx, bbox in enumerate(bboxes):

            logger.info('scanning wing {}: {}'.format(idx, bbox))

            (ymin, xmin, ymax, xmax) = bbox
            (ymin, xmin, ymax, xmax) = (ymin if direction[0] > 0 else ymax,
                                        xmin if direction[1] > 0 else xmax,
                                        ymin if direction[0] < 0 else ymax,
                                        xmin if direction[1] < 0 else xmax)

            # set oc so we have correct magnification
            set_optical_configuration(path_to_nis, ocs_detail if not isinstance(ocs_detail, list) else ocs_detail[0])

            # do autofocus -> move to wing center and focus
            if auto_focus_detail:
                x_center = (xmin + xmax) / 2
                y_center = (ymin + ymax) / 2
                set_position(path_to_nis, [x_center, y_center])
                do_autofocus(path_to_nis)

            wing_path = os.path.join(save_base_path, prefix + '_wing' + str(idx) + ND2_SUFFIX) 

            # only one optical configuration
            if not isinstance(ocs_detail, list):

                # set oc so we have correct magnification
                set_optical_configuration(path_to_nis, ocs_detail)

                # get resolution
                (xres, yres, siz, mag) = get_resolution(path_to_nis)
                fov = get_fov_from_res(get_resolution(path_to_nis))
                logger.debug('detail resolution: {}, {}, {}, {}'.format(xres, yres, siz, mag))
                logger.debug('fov: {}'.format(fov))

                # get fov
                fov_x = xres * siz / mag
                fov_y = yres * siz / mag

                # do not actually do the detail acquisition
                if dry_run_details:
                    continue

                # do a manual grid acquisition via multipoint nD acquisition -> has to be stitched afterwards
                if not stitched:

                    # we scan around current z -> get that
                    pos = get_position(path_to_nis)

                    # generate the coordinates of the tiles
                    grid, tilesX, tilesY, overlap = gen_grid(fov, [xmin, ymin], [xmax, ymax], 0.15, True, True, True)

                    for g in grid:
                        logger.debug('wing {}: will scan tile at {}'.format(idx-1, g))

                    nda = NDAcquisition(wing_path)
                    nda.set_z(int(z_range/2), int(z_range/2), int(z_step), z_drive)
                    nda.add_points(map(lambda x : (x[0], x[1], pos[2] - pos[3]), grid))
                    nda.prepare(path_to_nis)
                    nda.run(path_to_nis)

                # do NIS's scan large image -> stitching is performend in NIS
                else:
                    do_large_image_scan(path_to_nis, wing_path, xmax, xmin, ymin, ymax, 15, True)

            # multiple ocs -> we have to do nD acquisition
            else:

                # set to first oc so we have correct magnification
                set_optical_configuration(path_to_nis, ocs_detail[0])

                # get resolution
                (xres, yres, siz, mag) = get_resolution(path_to_nis)
                fov = get_fov_from_res(get_resolution(path_to_nis))
                logger.debug('detail resolution: {}, {}, {}, {}'.format(xres, yres, siz, mag))
                logger.debug('fov: {}'.format(fov))

                # get fov
                fov_x = xres * siz / mag
                fov_y = yres * siz / mag

                # generate the coordinates of the tiles
                grid, tilesX, tilesY, overlap = gen_grid(fov, [xmin, ymin], [xmax, ymax], 0.15, True, True, True)

                for g in grid:
                    logger.debug('wing {}: will scan tile at {}'.format(idx-1, g))

                # do not actually do the detail acquisition
                if dry_run_details:
                    continue

                # NB: we have multiple channels, so we have to do
                # manual grid acquisition via multipoint nD acquisition -> has to be stitched afterwards

                # we scan around current z -> get that
                pos = get_position(path_to_nis)

                # setup nD acquisition
                nda = NDAcquisition(wing_path)
                nda.set_z(int(z_range/2), int(z_range/2), int(z_step), z_drive)
                nda.add_points(map(lambda x : (x[0], x[1], pos[2] - pos[3]), grid))

                for oc in ocs_detail:
                    nda.add_c(oc)

                nda.prepare(path_to_nis)
                nda.run(path_to_nis)

            if tiff_export_detail:
                wing_out_dir = wing_path[:-len(ND2_SUFFIX)]
                if not os.path.exists(wing_out_dir):
                    os.makedirs(wing_out_dir)
                export_nd2_to_tiff(path_to_nis, wing_path, wing_out_dir)

            def copy_details():
                # copy to server mount
                _wing_path = wing_path
                _wing_out_dir = wing_out_dir
                _tilesX, _tilesY, _overlap = tilesX, tilesY, overlap
                copy_lock(_wing_path, server_path_local)
                if tiff_export_detail:
                    files = [os.path.join(_wing_out_dir, f) for f in os.listdir(_wing_out_dir) if os.path.isfile(os.path.join(_wing_out_dir, f))]
                    copy_lock_to_dir(files, os.path.join(server_path_local, _wing_out_dir.rsplit(os.sep)[-1]))

                remote_path = '/'.join([server_path_remote, _wing_out_dir.rsplit(os.sep)[-1] if tiff_export_detail else _wing_path.rsplit(os.sep)[-1]])

                with ServerProxy("http://eco-gpu:8001/") as proxy:
                    proxy.stitch([remote_path, _tilesX, _tilesY, _overlap], tiff_export_detail)

            copy_det_thread = threading.Thread(target=copy_details)
            threads.append(copy_det_thread)
            copy_det_thread.start()
            
            if progress_indicator is not None:
                progress_indicator.set_progress((idx+1)/len(bboxes))
                progress_indicator.set_status('scanning wing {}'.format(idx+1))
    
    except KeyboardInterrupt:
        logger.info('Interrupted by user, stopping...')
    
    except Exception:
        traceback.print_exc()
    
    finally:
        if progress_indicator is not None:
            progress_indicator.set_progress(1.0)
            progress_indicator.set_status('finishing copy to server')
            
        logger.info('Waiting for all copy threads to finish...')
        for t in threads:
            t.join()
        logger.info('Done.')



In [ ]:
###################
# set up the environment, nis, and image saving path
###################

# data save paths
path_to_nis = 'C:\\Program Files\\NIS-Elements\\nis_ar.exe'
save_base_path = 'C:\\Users\\Nikon\\Documents\\David\\tmpOverview'
save_server_path_local = 'Y:\\auto-test'
save_server_path_remote = '/data/wing-scanner/auto-test'

# location of the calibration files
calib_left = 'C:\\Users\\Nikon\\Documents\\David\\overview_calibrations\\left_260418.json'
calib_mid = 'C:\\Users\\Nikon\\Documents\\David\\overview_calibrations\\mid_260418.json'
calib_right = 'C:\\Users\\Nikon\\Documents\\David\\overview_calibrations\\right_260418.json'

# detail images z settings
z_drive='NIDAQ Piezo Z'
z_range = 30
z_step=2

# optical configurations to use
oc_overview = 'DIA4x'
ocs_detail = ['DIA10x', 'GFP 10x']

# plot size
%matplotlib inline
plt.rcParams['figure.figsize'] = [10,10]

In [ ]:
#################
# do the scans
#################

from ipywidgets import HBox, Checkbox, Text, BoundedFloatText, Button, FloatProgress, Label
from IPython.display import display

logging.basicConfig(format='%(asctime)s - %(levelname)s in %(funcName)s: %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

pos = get_position(path_to_nis)

left = HBox([Checkbox(description='image left slide'), Text(description='sample name:'),
             Checkbox(description='manual focus position'), BoundedFloatText(description='z', value=float(pos[2]), min=0, max=5000, step=10)])

mid = HBox([Checkbox(description='image middle slide'), Text(description='sample name:'),
             Checkbox(description='manual focus position'), BoundedFloatText(description='z', value=float(pos[2]), min=0, max=5000, step=10)])

right = HBox([Checkbox(description='image right slide'), Text(description='sample name:'),
             Checkbox(description='manual focus position'), BoundedFloatText(description='z', value=float(pos[2]), min=0, max=5000, step=10)])

status_main = HBox([FloatProgress(), Label()])
status_detail = HBox([FloatProgress(), Label()])

go = Button(description='GO')
status = WidgetProgressIndicator(status_detail.children[0], status_detail.children[1])

def onclick_go(btn):
    btn.disabled = True
    
    slide_left = left.children[1].value if left.children[0].value else None 
    slide_mid = mid.children[1].value if mid.children[0].value else None
    slide_right = right.children[1].value if right.children[0].value else None

    z_left = float(left.children[3].value) if left.children[2].value else None 
    z_mid = float(mid.children[3].value) if mid.children[2].value else None
    z_right = float(right.children[3].value) if right.children[2].value else None

    status_main.children[0].value = 0
    status_main.children[1].value = 'scanning left'
    
    do_scan_left  = slide_left != None
    if do_scan_left:
        logger.info('Scanning left scan.')
        do_scan(calib_left, oc_overview, ocs_detail, path_to_nis, save_base_path,
                slide_left, save_server_path_local, save_server_path_remote,
                manual_z_overview=z_left, z_drive=z_drive, z_range=z_range, z_step=z_step, progress_indicator=status )
    else:
        logger.info('Skipping left slide.')

    # mid slide
    
    status_main.children[0].value = 33
    status_main.children[1].value = 'scanning mid'
    
    do_scan_mid  = slide_mid != None
    if do_scan_mid:
        logger.info('Scanning mid scan.')
        do_scan(calib_left, oc_overview, ocs_detail, path_to_nis, save_base_path,
                slide_mid, save_server_path_local, save_server_path_remote,
                manual_z_overview=z_mid, z_drive=z_drive, z_range=z_range, z_step=z_step, progress_indicator=status)
    else:
        logger.info('Skipping middle slide.')

    status_main.children[0].value = 66
    status_main.children[1].value = 'scanning right'
    
    # right slide
    do_scan_right  = slide_right != None
    if do_scan_right:
        logger.info('Scanning right scan.')
        do_scan(calib_left, oc_overview, ocs_detail, path_to_nis, save_base_path,
                slide_right, save_server_path_local, save_server_path_remote,
                manual_z_overview=z_right, z_drive=z_drive, z_range=z_range, z_step=z_step, progress_indicator=status)
    else:
        logger.info('Skipping right slide.')
    
    status_main.children[0].value = 100
    status_main.children[1].value = 'Done'
    
    btn.disabled = False
    
go.on_click(onclick_go)
display(left)
display(mid)
display(right)
display(go)
display(status_main)
display(status_detail)

# various test code below

In [ ]:
#copy_lock(ov_path, server_path_local + (os.sep + 'overview') if separate_dirs_on_server else '')



In [ ]:
from skimage.transform import pyramid_gaussian

ov_path = 'C:\\Users\\Nikon\\Documents\\David\\tmpOverview\\DH_Overview_00x_overview.nd2'
img = read_bf(ov_path)   

pyr = [p for p in pyramid_gaussian(img, max_layer= 2, downscale = 2)]
img_ds = pyr[2]

In [ ]:
# get resolution, binning and fov
(xres, yres, siz, mag) = get_resolution(path_to_nis)
fov_x = xres * siz / mag
fov_y = yres * siz / mag
    
live_fmt, capture_fmt = get_camera_format(path_to_nis)
binning_factor = float(capture_fmt.split()[1].split('x')[0])

bboxes = detect_wings_simple(img, pixel_size=siz/mag*binning_factor, plot=True, layers=2)

In [ ]:
from skimage.color import label2rgb
image_label_overlay = label2rgb(np.zeros(img_ds.shape), image=ii)
plt.imshow(image_label_overlay)

In [ ]:
np.max(pyr[0])

ii = rescale_intensity(pyr[2], out_range=(0,1))
np.max(ii)

In [ ]:
a = 'abc'
a[:-1]